In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import load_iris, load_boston, load_breast_cancer, load_wine, load_digits
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

from tqdm import tqdm

%pylab inline

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Populating the interactive namespace from numpy and matplotlib


/home/basil/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Берем выборку

In [ ]:
# LETTER
letter = pd.read_csv('../data/letter.csv', header=None)
X = letter.drop(0, 1).values
target = pd.factorize(letter[0])[0]

In [ ]:
# POKER
# https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass.html
from sklearn.datasets import load_svmlight_file
data = load_svmlight_file('../data/poker')
X, target = data[0].toarray(), data[1].astype(int)
X, target = X[target <= 7], target[target <= 7]
pd.Series(target).value_counts()

In [ ]:
# Sensorless
# https://archive.ics.uci.edu/ml/datasets/dataset+for+sensorless+drive+diagnosis
from sklearn.datasets import load_svmlight_file
data = load_svmlight_file('../data/Sensorless.scale')
X, target = data[0].toarray(), data[1].astype(int)-1
X, _, target, _ = train_test_split(X, target, train_size=0.2, random_state=42, stratify=target)

pd.Series(target).value_counts()

In [ ]:
# 20NEWS
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups

twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
cv = TfidfVectorizer(max_features=100)
cv = CountVectorizer(max_features=100)

X = cv.fit_transform(twenty_train.data).toarray()
target = twenty_train.target

In [ ]:
# DIGITS
dataset = load_digits()
df = pd.DataFrame(dataset['data'])
target = dataset['target']

X = df.values

In [3]:
# MODEL
df = pd.read_csv('../data/model_16_16.csv')
target = df['target'].astype(int).values
del df['target']

X = df.values

In [4]:
# TRAIN/TEST/VALID SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42, stratify=target)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)
print(X_train.shape, X_test.shape, X_val.shape)

(896, 2) (320, 2) (384, 2)


In [5]:
BaseClassifier = LinearSVC
# BaseClassifier = LogisticRegression

In [6]:
subset_mid = np.linspace(20, 140, 13, dtype=int)
subset_small = np.linspace(20, 55, 8, dtype=int)
subset_big_rare = np.linspace(20, 200, 10, dtype=int)
subset_big = np.linspace(10, 200, 20, dtype=int)
subset_all = np.unique(np.concatenate((subset_big, subset_small)))

# N случайных дихотомий (без отбора)

In [ ]:
attempts_data = []
N_attempts = 10
l = np.unique(target).size
N = 30 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, BaseClassifier)
    for score_type in wtypes:
        weight_type = None
        preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)
#             print(score_type, weight_type, accuracy_score(preds, y_val))

In [ ]:
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

In [ ]:
# делаем общее многозначное
attempts_data = []

subset = np.linspace(20, 55, 8, dtype=int)
for N in subset_all: # кол-во дихотомий
    N_attempts = 10
    l = np.unique(target).size
    wtypes = [None, 'accuracy', 'f1', 'confusion_list']
    for i in tqdm(range(N_attempts)):
        accs = []
        code_matrix = functions.make_random_dichs(l, N)
        print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
        dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                                 X_test, y_test, BaseClassifier)
        for score_type in wtypes:
            weight_type = None
            preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
            acc = accuracy_score(preds, y_val)
            accs.append(acc)
        accs.append(num_real_dich)
        attempts_data.append(accs)
    #             print(score_type, weight_type, accuracy_score(preds, y_val))


In [ ]:
cols = ['ecoc', 'accuracy', 'f1', 'confusion_list', 'num_real_dich']
df_attempts = pd.DataFrame(attempts_data, columns=cols)

gb = df_attempts.groupby('num_real_dich').agg(['mean', 'std'])
gb.columns = ['_'.join(col).strip() for col in gb.columns.values]
gb = gb.reset_index()
gb

# Отбор зазором

In [ ]:
attempts_data = []
N_attempts = 10
l = np.unique(target).size
N = 100 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
#     print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, BaseClassifier)
    for score_type in wtypes:
        preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, 
                                      code_matrix, score_type, weight_type=-1)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)

In [ ]:
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

In [ ]:
# делаем общее многозначное
attempts_data = []
subs = [220, 240, 280, 300, 350, 400]
for N in subs: # кол-во дихотомий
    N_attempts = 10
    l = np.unique(target).size
    wtypes = [None, 'accuracy', 'f1', 'confusion_list']
    for i in tqdm(range(N_attempts)):
        accs = []
        code_matrix = functions.make_random_dichs(l, N)
    #     print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
        dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                                 X_test, y_test, BaseClassifier)
        for score_type in wtypes:
            preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, 
                                          code_matrix, score_type, weight_type=-1)
            acc = accuracy_score(preds, y_val)
            accs.append(acc)
        accs.append(num_real_dich)
        attempts_data.append(accs)

In [ ]:
cols = ['ecoc', 'accuracy', 'f1', 'confusion_list', 'num_real_dich']
df_attempts = pd.DataFrame(attempts_data, columns=cols)
df_attempts['gb_index'] = (df_attempts.index / N_attempts).astype(int)

gb = df_attempts.groupby('gb_index').agg(['mean', 'std'])
gb.columns = ['_'.join(col).strip() for col in gb.columns.values]
gb = gb.reset_index()
gb

# Дихотомии где остались лучшие по критерию

In [ ]:
attempts_data = []
N_attempts = 10
l = np.unique(target).size
N = 300 # кол-во дихотомий
num_real_dich = 100 # кол-во дихотомий которые оставляем
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, BaseClassifier)
    
    metric_map = np.array([d['f1'] for d in dich_classifiers])
    top_dich = np.sort(np.argsort(metric_map)[-num_real_dich:])
    dich_classifiers = [d for i,d in enumerate(dich_classifiers) if i in top_dich]
    code_matrix = code_matrix.T[top_dich].T
    
    for score_type in wtypes:
        weight_type = None
        preds, _ = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)
#             print(score_type, weight_type, accuracy_score(preds, y_val))

In [ ]:
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

In [ ]:
%%time
attempts_data = []
N = 400
l = np.unique(target).size
for num_real_dich in subset_mid: # кол-во дихотомий
    N_attempts = 10
    wtypes = [None, 'accuracy', 'f1', 'confusion_list']
    for i in tqdm(range(N_attempts)):
        accs = []
        code_matrix = functions.make_random_dichs(l, N)
        print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
        dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                                 X_test, y_test, BaseClassifier)

        metric_map = np.array([d['f1'] for d in dich_classifiers])
        top_dich = np.sort(np.argsort(metric_map)[-num_real_dich:])
        dich_classifiers = [d for i,d in enumerate(dich_classifiers) if i in top_dich]
        code_matrix = code_matrix.T[top_dich].T

        for score_type in wtypes:
            weight_type = None
            preds, _ = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
            acc = accuracy_score(preds, y_val)
            accs.append(acc)
        accs.append(num_real_dich)
        attempts_data.append(accs)

In [ ]:
cols = ['ecoc', 'accuracy', 'f1', 'confusion_list', 'num_real_dich']
df_attempts = pd.DataFrame(attempts_data, columns=cols)

gb = df_attempts.groupby('num_real_dich').agg(['mean', 'std'])
gb.columns = ['_'.join(col).strip() for col in gb.columns.values]
gb = gb.reset_index()
gb

# Локальный метод оптимизации

In [ ]:
def score_function(cur_dich, code_matrix):
    # cluster
    if cur_dich.max() == cur_dich.min(): #trivial dich
        return -inf
    target = np.array([cur_dich[i] for i in y_train])
    return -functions.cluster_score(X_train, target, score_type='trace_w')

In [ ]:
def score_function(cur_dich, code_matrix):
    # accuracy
    X = X_train
    y = np.array([cur_dich[i] for i in y_train])
    if y.max() == y.min(): #trivial dich
        return 0
    clf = LogisticRegression()
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    y_true = np.array([cur_dich[i] for i in y_test])
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [8]:
def score_function(cur_dich, code_matrix):
    # f1
#     X = X_train
#     y = np.array([cur_dich[i] for i in y_train])
# ВОЗМОЖНО ЗДЕСЬ БЫЛО БОЛЬШОЕ ПЕРЕОБУЧЕНИЕ!!!
    X = X_test
    y = np.array([cur_dich[i] for i in y_test])
    if y.max() == y.min(): #trivial dich
        return 0
    clf = LogisticRegression()
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    y_true = np.array([cur_dich[i] for i in y_test])
    accuracy = f1_score(y_true, y_pred)
    return accuracy

In [ ]:
# матрица неточностей
from sklearn.metrics import confusion_matrix
def score_function(cur_dich, code_matrix):
    if cur_dich.max() == cur_dich.min():
        return 0
    if code_matrix is None:
        return 0
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, BaseClassifier)
    preds, num_real_dich = functions.predict_all(X_test, dich_classifiers, code_matrix, None, None)
    C = confusion_matrix(y_test, preds)
    D = (cur_dich+cur_dich.T) % 2
    q = (C*D).sum()
    return q

In [ ]:
attempts_data = []
N_attempts = 10
l = np.unique(target).size
N = 10 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    
    code_matrix = functions.make_code_matrix_local(l, N, score_function, 0)
    while code_matrix.sum(axis=0).max() == l or code_matrix.sum(axis=0).min() == 0:
        code_matrix = functions.make_code_matrix_local(l, N, score_function, 0)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, BaseClassifier)
    for score_type in wtypes:
        weight_type = None
        preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)

In [ ]:
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

In [9]:
attempts_data = []
N_attempts = 10
l = np.unique(target).size
wtypes = [None, 'accuracy', 'f1', 'confusion_list']

for N in subset_mid: 
    for i in tqdm(range(N_attempts)):
        accs = []
        code_matrix = functions.make_code_matrix_local(l, N, score_function, 0)
        while code_matrix.sum(axis=0).max() == l or code_matrix.sum(axis=0).min() == 0:
            code_matrix = functions.make_code_matrix_local(l, N, score_function, 0)
        print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
        dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                                 X_test, y_test, BaseClassifier)
        for score_type in wtypes:
            weight_type = None
            preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
            acc = accuracy_score(preds, y_val)
            accs.append(acc)
        accs.append(num_real_dich)
        attempts_data.append(accs)

 10%|█         | 2/20 [00:00<00:01, 11.99it/s]/home/basil/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)

100%|██████████| 20/20 [00:01<00:00, 11.74it/s]

Training dich classifiers: 100%|██████████| 20/20 [00:00<00:00, 206.54it/s]

Code Matrix shape == (16,20)


Training dich classifiers: 100%|██████████| 20/20 [00:00<00:00, 283.84it/s]

Code Matrix shape == (16,20)


100%|██████████| 20/20 [00:01<00:00, 12.80it/s]

Training dich classifiers: 100%|██████████| 20/20 [00:00<00:00, 233.50it/s]

...


Training dich classifiers:  19%|█▊        | 26/140 [00:00<00:00, 255.82it/s]

Code Matrix shape == (16,140)



100%|██████████| 140/140 [00:11<00:00, 12.61it/s]

Training dich classifiers:  21%|██▏       | 30/140 [00:00<00:00, 290.32it/s]

Code Matrix shape == (16,140)



100%|██████████| 10/10 [02:48<00:00, 16.83s/it]/140 [00:00<00:00, 178.13it/s]


In [10]:
cols = ['ecoc', 'accuracy', 'f1', 'confusion_list', 'num_real_dich']
df_attempts = pd.DataFrame(attempts_data, columns=cols)

gb = df_attempts.groupby('num_real_dich').agg(['mean', 'std'])
gb.columns = ['_'.join(col).strip() for col in gb.columns.values]
gb = gb.reset_index()
gb

,num_real_dich,ecoc_mean,ecoc_std,accuracy_mean,accuracy_std,f1_mean,f1_std,confusion_list_mean,confusion_list_std
0,20,0.809896,0.027423,0.835156,0.022504,0.831771,0.024620,0.839844,0.024560
1,30,0.831510,0.018293,0.846615,0.017724,0.840104,0.021446,0.854427,0.014181
2,40,0.845052,0.011981,0.857812,0.013061,0.855208,0.010856,0.855990,0.008685
3,50,0.851823,0.013584,0.860938,0.013403,0.859635,0.012486,0.861458,0.011762
4,60,0.844792,0.017283,0.851562,0.015234,0.849219,0.013361,0.864844,0.009223
5,70,0.849479,0.009882,0.855990,0.010984,0.852604,0.011915,0.863281,0.007289
6,80,0.849219,0.010804,0.855729,0.014894,0.853125,0.014484,0.864062,0.010474
7,90,0.848177,0.010201,0.853646,0.011371,0.852604,0.008784,0.868490,0.007885
8,100,0.857812,0.011788,0.861458,0.013824,0.861198,0.012582,0.870052,0.010663
9,110,0.854427,0.010157,0.861458,0.007242,0.861719,0.007513,0.869531,0.008806
